In [1]:
# Welcome. Use this program to design oligos for making single sgRNA plasmids. 
# DO NOT use this to make oligos for pooled sgRNA  libraries. 
# Those primers are slightly different


In [2]:
import os
cwd = os.getcwd()
cwd
#this is just to help you find where your excel files are
#you will need to download the Calabrese library if you are designing CRISPRa sgRNA and
#the Dolcetto library if you are designing CRISPRi

'/Users/willboyd/Documents/Lab/Jupyter notebooks/Pioneer Factor sgRNA Library'

In [3]:
#For the new user, there are only 2 parts you need to change for each run
#the genes you want to target and the CRISPRi/a library you are taking the sgRNA sequence from

#Type the names of the Genes you are interested in targeting here

Target_Genes = ['HNF4A', 'FOXA2', 'CDX2', 'GATA6'] 

#Load the CRISPRi/a library you are taking the sgRNA sequence from here

import pandas as pd
df = pd.read_excel("/Users/willboyd/Documents/Lab/Jupyter notebooks/Pioneer Factor sgRNA Library/Calabrese Library A.xlsx")

# Load data from Genomewide sgRNA Library file
#Make sure that the sgRNA are coming from the Calabrese Library if you are looking for CRISPRa sgRNA
# and from the Dolcetto if you are looking for CRISPRi sgRNA


#if this part works you should be able to run the rest with no issues
#also make sure to restart the kernel for every run, im not good at coding

In [4]:

# Convert the 'Annotated Gene Symbol' column to strings
df['Annotated Gene Symbol'] = df['Annotated Gene Symbol'].astype(str)

# Sort the DataFrame by Annotated Gene Symbol
df = df.sort_values(by="Annotated Gene Symbol").reset_index(drop=True)

# Dictionary to keep track of occurrences of each Annotated Gene Symbol
gene_count = {}

# List to store the modified Annotated Gene Symbols
modified_gene_symbols = []

# Iterate through each row to modify Annotated Gene Symbols
for gene_symbol in df['Annotated Gene Symbol']:
    if gene_symbol not in gene_count:
        gene_count[gene_symbol] = 1
    else:
        gene_count[gene_symbol] += 1
    
    # Append the suffix to the Annotated Gene Symbol
    modified_gene_symbols.append(f"{gene_symbol}-{gene_count[gene_symbol]}")

# Add the modified Annotated Gene Symbols to the DataFrame
df['Modified Annotated Gene Symbol'] = modified_gene_symbols

# Select relevant columns and rename for clarity
organized_df = df[['Modified Annotated Gene Symbol', 'Barcode Sequence', 'Annotated Gene Symbol']]




In [5]:

# Create a filter that matches any of the words in the 'Annotated Gene Symbol' column
filter_mask = organized_df['Annotated Gene Symbol'].isin(Target_Genes)

# Filter the DataFrame
filtered_df = df[filter_mask]

filtered_df =filtered_df[['Modified Annotated Gene Symbol', 'Annotated Gene Symbol', 'Barcode Sequence']]


In [6]:
# Copy each row
copied_fdf = filtered_df.copy()
# Append "-fwd" to the original Modified Annotated Gene Symbol and "rev" to the copied Modified Annotated Gene Symbol
copied_fdf['Modified Annotated Gene Symbol'] = copied_fdf['Modified Annotated Gene Symbol'] + "-rev"
filtered_df['Modified Annotated Gene Symbol'] = filtered_df['Modified Annotated Gene Symbol'] + "-fwd"

#Make the rev barcode sequence the reverse complement of the fwd
complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
copied_fdf['Barcode Sequence'] = copied_fdf['Barcode Sequence'].apply(lambda x: ''.join(complement[base] for base in x[::-1]))

#append a CACCG to the front of the fwd oligo to add the sticky hands and the U6 start codon "G"
#append a AAAC to the front and C to the back of the rev oligo to add the sticky hands and start codon
filtered_df['Barcode Sequence'] = 'CACCG' + filtered_df['Barcode Sequence'].astype(str)
copied_fdf['Barcode Sequence'] = 'AAAC' + copied_fdf['Barcode Sequence'].astype(str) + "C"

#Combine, organize and export the finalized oligo sequences
final_df = pd.concat([filtered_df, copied_fdf], ignore_index=True)
final_df = final_df[['Modified Annotated Gene Symbol', 'Annotated Gene Symbol', 'Barcode Sequence']]
final_df = final_df.sort_values(by="Modified Annotated Gene Symbol").reset_index(drop=True)
final_df.to_excel("primer_data.xlsx", index=False)

#check the product here
final_df


,Modified Annotated Gene Symbol,Annotated Gene Symbol,Barcode Sequence
0,CDX2-1-fwd,CDX2,CACCGAATGCAAATTATGTTTCGAG
1,CDX2-1-rev,CDX2,AAACCTCGAAACATAATTTGCATTC
2,CDX2-2-fwd,CDX2,CACCGAGGGGAGGGAGGCTCGTGAG
3,CDX2-2-rev,CDX2,AAACCTCACGAGCCTCCCTCCCCTC
4,CDX2-3-fwd,CDX2,CACCGGGGGAGGGAGGCTCGTGAGA
5,CDX2-3-rev,CDX2,AAACTCTCACGAGCCTCCCTCCCCC
6,FOXA2-1-fwd,FOXA2,CACCGAGAGGGAGCGCGAGAGAGGG
7,FOXA2-1-rev,FOXA2,AAACCCCTCTCTCGCGCTCCCTCTC
8,FOXA2-2-fwd,FOXA2,CACCGCTCTCGCGCTCCCTCTCCCT
9,FOXA2-2-rev,FOXA2,AAACAGGGAGAGGGAGCGCGAGAGC
